# Walk Score API

cd <localdir>
git init
git add .
git commit -m 'message'
git remote add origin <url>
git push -u origin master

Example request:

```python
response = get_walkscore(address='1119 8th Avenue Seattle WA 98101',lat='47.6085',lon='-122.3295')
response.json()
```

Example output:

```python
{'status': 1,
 'walkscore': 99,
 'description': "Walker's Paradise",
 'updated': '2020-11-18 19:42:23.803841',
 'logo_url': 'https://cdn.walk.sc/images/api-logo.png',
 'more_info_icon': 'https://cdn.walk.sc/images/api-more-info.gif',
 'more_info_link': 'https://www.redfin.com/how-walk-score-works',
 'ws_link': 'https://www.walkscore.com/score/1119-208th-20Avenue-20Seattle-20WA-2098101/lat=47.6085/lng=-122.3295/?utm_source=peti.work&utm_medium=ws_api&utm_campaign=ws_api',
 'help_link': 'https://www.redfin.com/how-walk-score-works',
 'snapped_lat': 47.6085,
 'snapped_lon': -122.3295}
```

In [1]:
import requests as r
def get_walkscore(lat, lon, address):
    api_key = "18462ed72359fe80c186f0796f512165"
    url = "https://api.walkscore.com/score?format=json"
    addie = address.replace(" ", "%20")
    params = {
        'address':addie,
        'lat':lat,
        'lon':lon,
        'wsapikey':api_key
    }
    response = r.get(url=url,params=params)
    return response

In [13]:
response = get_walkscore(address='1119 8th Avenue Seattle WA 98101',lat='47.6085',lon='-122.3295')

In [14]:
response.json()

{'status': 1,
 'walkscore': 99,
 'description': "Walker's Paradise",
 'updated': '2020-11-18 19:42:23.803841',
 'logo_url': 'https://cdn.walk.sc/images/api-logo.png',
 'more_info_icon': 'https://cdn.walk.sc/images/api-more-info.gif',
 'more_info_link': 'https://www.redfin.com/how-walk-score-works',
 'ws_link': 'https://www.walkscore.com/score/1119-208th-20Avenue-20Seattle-20WA-2098101/lat=47.6085/lng=-122.3295/?utm_source=peti.work&utm_medium=ws_api&utm_campaign=ws_api',
 'help_link': 'https://www.redfin.com/how-walk-score-works',
 'snapped_lat': 47.6085,
 'snapped_lon': -122.3295}

In [17]:
import pandas as pd

In [22]:
df = pd.read_html('walkscore.html')[0]

In [23]:
df

,City,State,Walk Score,Transit Score,Bike Score,Population
0,New York,NY,88.3,84.3,70.0,8175133
1,San Francisco,CA,87.4,80.3,72.2,805235
2,Jersey City,NJ,86.8,70.6,70.3,247597
3,Boston,MA,82.0,72.5,70.5,617594
4,Philadelphia,PA,78.8,66.8,67.4,1526006
...,...,...,...,...,...,...
103,Montgomery,AL,26.1,16.4,38.3,205764
104,Jacksonville,FL,25.7,23.0,42.7,821784
105,Winston-Salem,NC,23.4,--,29.2,229617
106,Fayetteville,NC,21.4,--,33.2,200564


# Weather API

weatherapi.com

```python
## Adding hours or minutes or seconds to datetime
from datetime import datetime, timedelta
 
## Original datetime
datetime_original = datetime(year=2006, month=11, day=23)
print("\nOriginal date: ", datetime_original, "\n")
 
## Adding Hours
hours_to_add = 12
datetime_new = datetime_original + timedelta(hours = hours_to_add)
print("After adding hours: ", datetime_new, "\n")
 
## Adding Minutes
minutes_to_add = 45
datetime_new = datetime_new + timedelta(minutes = minutes_to_add)
print("After adding minutes: ", datetime_new, "\n")
 
## Adding Seconds
seconds_to_add = 33
datetime_new = datetime_new + timedelta(seconds = seconds_to_add)
print("After adding seconds: ", datetime_new, "\n")
 
## Adding Microseconds
microseconds_to_add = 12345
datetime_new = datetime_new + timedelta(microseconds = microseconds_to_add)
print("After adding microseconds: ", datetime_new, "\n")
```

In [178]:
import requests as r
import numpy as np
import datetime as dt

class weatherAnalysis():
    def __init__(self, location, type_='future'):
        '''
        Initiates weather api
        
        input
        ----
        location: str
            Zip code, ip address, lat/lon, city name
        type_: str
            current or future or history
        '''
        api_key = "4216ca74f3d1478dbbd154714212001"

        params = {
            'key':api_key,
            'q':location
        }

        if type_ == 'current':
            url = 'https://api.weatherapi.com/v1/current.json'
        elif type_ == 'future':
            url = 'https://api.weatherapi.com/v1/forecast.json'
            params['days'] = 3 # max forecast for free account
        elif type_ == 'history':
            url = 'https://api.weatherapi.com/v1/history.json'
            self.history(url, params)
            return
            
        response = r.get(url = url, params = params)
        self.response = response.json()

        if type_ == 'future':
            self.forecast()
        
    def date_lister(self):
        '''
        Returns list of 7 properly formatted dates
        '''
        historic_dates = []
        
        for i in range(1,8):
            new_date = dt.datetime.now() - dt.timedelta(days=i) # max history for free account
            historic_dates.append(new_date.strftime('%Y-%m-%d'))
        
        return historic_dates
    
    def forecast(self):
        '''
        Gets the forecast, calculates mean temp
        '''
        temps = []
        all_forecast = self.response['forecast']['forecastday']
        for day in all_forecast:
            temp = day['day']['avgtemp_f'] # get avg temp for day
            temps.append(temp)
        
        self.all_temps = temps
        self.mean_temp = np.mean(temps)
        
    def history(self, url, params):
        '''
        Gets historic weather, calculates mean temp
        
        input
        -----
        url: str
            Api url
        params: dict
            Constructed params, without date
            
        output
        ------
        self.historic: pd.DataFrame
            Dataframe of dates and temp in F
        '''
        temps = []
        date_list = self.date_lister()
        
        all_responses = []
        for date in date_list:
            params['dt'] = date
            response = r.get(url=url, params=params)
            all_responses.append(response.json())
        
        extracted = {}
        for i in range(len(all_responses)):
            selected = all_responses[i]['forecast']['forecastday'][0] # all info for selected day
            day = selected['day'] # temp stats for the day    
            extracted[selected['date']] = [day['avgtemp_f'],day['mintemp_f'],day['maxtemp_f']]
            
        df = pd.DataFrame(extracted).T.reset_index()
        df.columns = ['date','avg_temp_f','min_temp_f','max_temp_f']
        self.historic = df 

In [179]:
w = weatherAnalysis(44224, type_='history')

In [181]:
w.historic.avg_temp_f.mean()

32.17142857142857